In [ ]:
%pip install prisma

In [ ]:
!prisma db push

In [ ]:
import requests

requests.packages.urllib3.disable_warnings()
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

from bs4 import BeautifulSoup as bs

In [ ]:
author_name = "roberto-benigni"
url = f"https://www.frasicelebri.it/frasi-di/{author_name}"

In [ ]:
try:
    response = requests.get(url, verify=False)
    response.raise_for_status()  # Raise an exception for non-200 status codes
except requests.exceptions.RequestException as e:
    print(f"Error fetching page: {e}")

soup = bs(response.text, "html.parser")

last_page_number = soup.select_one(".fc-pagination ul li.last-page a").text

In [ ]:
author_pages = [f"{url}/?page={i}" for i in range(1, int(last_page_number))]
pages_text = [requests.get(page, verify=False).text for page in author_pages]

In [ ]:
blockquotes = [(bs(page).select("blockquote.clearfix")) for page in pages_text]

In [ ]:
for blockquote in blockquotes:
    quotes = [(quote.select_one("span.whole-read-more").text) for quote in blockquote]

In [ ]:
quotes

### insert quotes in database


In [ ]:
import asyncio

from prisma import Prisma

db = Prisma()
await db.connect()

In [39]:
author_name = "roberto-"
# Check if author  exist in the website
author = await db.author.find_unique(
    where={"name": author_name},
    include={
        "quotes": True,
    },
)


if author:
    # pr author existe create author in db
    print("autore presente")
else:
    print(f"{author_name} non esiste")

roberto- non esiste


In [ ]:
author = await db.author.create({"name": author_name})

In [31]:
quote = await db.quote.upsert(
    where={
        "text": quotes[2],
    },
    data={
        "create": {
            "text": quotes[2],
        },
        "update": {},
    },
)
quote.text

'“- Arbore: Mi hanno parlato di un toscano, mi pare un fiorentino, che vende un sacco di milioni e milioni e milioni di copie da tanto tempo in tutto il mondo con un libro che parla pure di morti, di aldilà, di cose...di cadaveri...di inferno, purgatorio...- Benigni: Dante!- Arbore: No! Un nome più... più.. Alghero.. una cosa così...- Benigni:...” (continua)'

In [ ]:
quote = await db.quote.create(
    {
        "text": quote,
        "author": {
            "connect": {
                "id": author.id,
            }
        },
    }
)